In [ ]:
import json
import os
import re
import traceback

import math
import scipy
import scipy.stats

import pyzipcode
import hashlib

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown
from IPython.display import HTML
from tqdm import tqdm
import fuzzywuzzy
import fuzzywuzzy.fuzz

import sqlite3
import pandas as pd
import nltk

import itertools
import collections
import functools
import collections
import random


In [ ]:
yc_reviews = pd.read_pickle("../data/pickles/yelpchi_reviews_unmatched.pkl")
yc_updated_reviews = pd.read_pickle("../data/pickles/yelpchi_updated_reviews_unmatched.pkl")
yc_businesses = pd.read_pickle("../data/pickles/yelpchi_businesses_unmatched.pkl")

# Let's try to match reviews across the datasets

In [ ]:
matches = {}
matches_sc = {}

multiple_matches = {}

In [ ]:
yc_updated_reviews

In [ ]:
#yc_updated_reviews_indexed = yc_updated_reviews.reset_index().rename(columns={"index": "reviewID"}).set_index(["businessID", "reviewHash", "reviewHashNoLF", "reviewID"])

In [ ]:
#yc_updated_reviews_indexed

In [ ]:
hash_to_id = collections.defaultdict(set)
yc_reviews_at_time = yc_updated_reviews[yc_updated_reviews.date <= yc_reviews.date.max()]
out_of = len(yc_reviews_at_time)
for rowid, row in tqdm(yc_reviews_at_time.iterrows(),total=out_of):
    hash_to_id[row.reviewHashStripped].add(rowid)
    hash_to_id[row.reviewHashBOW].add(rowid)
    hash_to_id[row.reviewHashBOWNoLF].add(rowid)

In [ ]:
ct = 0
out_of = len(yc_reviews)


no_matches = []

for rowid,row in tqdm(yc_reviews.iterrows(), total = out_of):
    ct += 1
    
    if rowid in matches or rowid in multiple_matches:
        continue
    
    content1 = row.reviewContent
    business_id = row.businessID
    
    
    exact_matches = pd.concat([yc_updated_reviews.loc[hash_to_id[row.reviewHashBOW]], yc_updated_reviews.loc[hash_to_id[row.reviewHashStripped]]]).drop_duplicates()
    
    
    #new_reviews = yc_updated_reviews[yc_updated_reviews.businessID == business_id]
    #exact_matches = new_reviews[new_reviews.reviewHash == row.reviewHash]
    #exact_matches_LF = new_reviews[new_reviews.reviewHash == row.reviewHash]
    #exact_matches_no_LF = new_reviews[new_reviews.reviewHashNoLF == row.reviewHash]
#     new_reviews = yc_updated_reviews_indexed.loc[business_id]
#     try:
#         exact_matches_no_LF = new_reviews.loc[(slice(None),row.reviewHash)]
#     except:
#         exact_matches_no_LF = pd.DataFrame()
#     try:
#         exact_matches_LF = new_reviews.loc[(row.reviewHash,slice(None))]
#     except:
#         exact_matches_LF = pd.DataFrame()
    #exact_matches = pd.concat([exact_matches_LF,exact_matches_no_LF])
    
    if len(exact_matches) == 0:
        no_matches.append(rowid)
        
        continue
        
        closest = None
        closest_sc = 0
        
        new_reviews = yc_updated_reviews[yc_updated_reviews.businessID == business_id]
        
        for rowid2, row2 in new_reviews.iterrows():
            content2 = row2.content
            score = compare_reviews(content1,content2)
            if score > closest_sc:
                closest = rowid2
                closest_sc = score
                if score == 1:
                    break
        if closest_sc < 90:
            continue
    elif len(exact_matches) == 1:
        closest = exact_matches.index[0]
        closest_sc = 1
    else:
        for matchid, match in exact_matches.iterrows():
            try:
                reviewer_name = yc_reviewers.loc[row.reviewerID].name
            except KeyError:
                reviewer_name = None
            if match.date == row.date or match.user_name == reviewer_name:
                closest = matchid
                closest_sc = 1
                break
        multiple_matches[rowid] = row,exact_matches
        continue
        
    matches[rowid] = closest
    matches_sc[rowid] = closest_sc

In [ ]:
key = random.choice(list(multiple_matches.keys()))
print(key)
display(pd.DataFrame(multiple_matches[key][0]).transpose())
display(multiple_matches[key][1])

In [ ]:
yc_reviews["matchID"] = pd.Series([matches[rowid] if rowid in matches else None for rowid in yc_reviews.index.values])
yc_reviews["match_score"] = pd.Series([matches_sc[rowid] if rowid in matches_sc else None for rowid in yc_reviews.index.values])

#### Are there obvious false positives?

In [ ]:
def username_matches(row):
    if math.isnan(row.matchID):
        print("Null match ID")
        return None
    match = yc_updated_reviews.loc[row.matchID]
    try:
        username = yc_reviewers.loc[row.reviewerID].name
    except KeyError:
        print(f"No name for {row.reviewerID}")
        return None
    return match.user_name == username

def date_matches(row):
    if math.isnan(row.matchID):
        return None
    match = yc_updated_reviews.loc[row.matchID]
    return match.date == row.date

def business_matches(row):
    if math.isnan(row.matchID):
        return None
    match = yc_updated_reviews.loc[row.matchID]
    return match.businessID == row.businessID

display(pd.DataFrame(yc_reviews.apply(date_matches, axis=1)).groupby(0).size())
display(pd.DataFrame(yc_reviews.apply(business_matches, axis=1)).groupby(0).size())
display(pd.DataFrame((yc_reviews.apply(date_matches, axis=1) | yc_reviews.apply(business_matches, axis=1))).groupby(0).size())

#### Are there double matches?

In [ ]:
pd.DataFrame(yc_reviews.groupby("matchID").size()).groupby(0).size()

Very few

In [ ]:
reject_reviews = yc_reviews.loc[no_matches]
reject_reviews.groupby("flagged").size(), yc_reviews[yc_reviews.matchID.notnull()].groupby("flagged").size(), yc_reviews.groupby("flagged").size()

In [ ]:
matched_updated = yc_updated_reviews.loc[matches.values()]
yc_reviews_at_time = yc_updated_reviews[yc_updated_reviews.date < yc_reviews.date.max()]
unmatched_updated = yc_reviews_at_time.loc[set(yc_reviews_at_time.index.values) - set(matches.values()) ]

In [ ]:
unmatched_updated.groupby("flagged").size(), matched_updated.groupby("flagged").size(), yc_reviews_at_time.groupby("flagged").size()

In [ ]:
reject_sample = reject_reviews.sample(100)

In [ ]:
max_width = max((len(i) for i in reject_sample.reviewerID))
x=0
print("\n".join(f"{str((x := x+1)).rjust(3,' ')}. {s.reviewerID}|{s.reviewContent[:100].ljust(100,' ')}|{i}" for i,s in reject_sample.iterrows()))

#### Mismatches:
12: Review was modified

18: Review exists but was not collected


In [ ]:
s1 = reject_reviews.loc[4310]
review1 = s1.reviewContent
print(s1.businessID)
print(review1)

display(Markdown("---"))

sel = None
for review in review1:
    selO = yc_updated_reviews.content.str.startswith(str(review[:50]))
    if sel is None:
        sel = selO
    else:
        sel |= selO
s2 = yc_updated_reviews[sel]
review2 = s2.content
print(review2)

#### Review 54217 is a strange case -- manual inspection reveals the review does not exist flipping through the pages, but it can be found by searching

In [ ]:
sns.distplot(unmatched_updated.groupby("businessID").size().sort_values())

In [ ]:
for i, m in multiple_matches.items():
    display(i)
    for x in m:
        display(x)
    display(Markdown("---"))

# Add features to business files

In [ ]:
#yc_reviews.matchID

#yc_updated_reviews

def get_change_type(row):
    oldFil = row.flagged
    if math.isnan(row.matchID):
        newFil = "X"
    else:
        match = yc_updated_reviews.loc[row.matchID]
        newFil = match.flagged
    
    return f"change_{oldFil}{newFil}"

def get_changes_columns(row):
    old_reviews = yc_reviews[yc_reviews.businessID == row.name]
    return pd.DataFrame(old_reviews.apply(get_change_type,axis=1)).groupby(0).size()
    
change_columns = yc_businesses.apply(get_changes_columns,axis=1).fillna(0)

In [ ]:
change_columns.sum()

In [ ]:
def normalize_changes(row):
    total = sum(row)
    return pd.Series([row[colname]/total for colname in row.index.values], index=[f"{colname}_ratio" for colname in row.index.values])
    
        
change_columns_perc = change_columns.apply(normalize_changes,axis=1)

In [ ]:
change_columns_perc.sort_values("change_NY_ratio")

In [ ]:
yc_businesses_updated = pd.concat([yc_businesses, change_columns, change_columns_perc],axis=1)

# Save 

In [ ]:
yc_reviews.to_pickle("../data/pickles/yelpchi_reviews.pkl")
yc_updated_reviews.to_pickle("../data/pickles/yelpchi_updated_reviews.pkl")
yc_businesses_updated.to_pickle("../data/pickles/yelpchi_businesses.pkl")